In [17]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
import ssl
import numpy as np
from scipy import stats
ssl._create_default_https_context = ssl._create_unverified_context

# Refer to cifarDataPrep notebook for step-by-step explanation of data preparation
# Load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Preprocess the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert the target labels to one-hot encoding
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)



In [18]:
# Build the baseline model
base_model = tf.keras.Sequential([
    # Convolutional layers are crucial for capturing spatial patterns in images. The chosen configuration 
    # consists of two pairs of Conv2D layers. The first pair has 32 filters, and the second pair has 64 filters. 
    # Each Conv2D layer uses a 3x3 filter size, which is a common choice. These layers apply convolution operations 
    # to the input image, extracting relevant features and learning representations that are useful for classification.
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    # Max pooling is used to downsample the feature maps obtained from the convolutional layers. It helps reduce the
    # spatial dimensions while retaining the most important information. In this model, MaxPooling2D layers with a 
    # 2x2 pool size are added after each pair of Conv2D layers to progressively reduce the spatial dimensions.
    tf.keras.layers.MaxPooling2D((2, 2)),
    # Dropout layers help prevent overfitting by randomly setting a fraction of input units to 0 during training. 
    # The dropout rate of 0.25 is chosen in this model, meaning 25% of the units are randomly dropped during each training step. 
    # Dropout acts as a regularization technique, promoting the model's ability to generalize and reducing the likelihood of overfitting.
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    # The Flatten layer is used to convert the 2D feature maps from the previous layer into a 1D feature vector. 
    # This flattening operation is necessary to connect the convolutional layers to the subsequent dense (fully connected) layers.
    tf.keras.layers.Flatten(),
    # Dense layers are fully connected layers where each neuron is connected to every neuron in the previous layer. In this model, 
    # there is one dense layer are added with 512 units. This layer provides higher-level abstractions by combining features learned 
    # from the convolutional layers. The use of ReLU activation in the dense layer helps introduce non-linearity to the model.
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # The final dense layer has the number of units equal to the number of classes in the dataset (10 in this case). 
    # The softmax activation function is used to produce probability scores for each class, enabling the model to make predictions.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])



In [19]:
from tensorflow.keras import regularizers

# Building a transfer learning model to improve upon the baseline model
# Load the VGG16 model pre-trained on ImageNet
# By setting include_top=False, we exclude the fully connected layers at the top of the VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the pre-trained layers
for layer in range(11):
    vgg_model.layers[layer].trainable = False

vgg16_model = tf.keras.Sequential([
    vgg_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [20]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [21]:
vgg16_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_6 (Flatten)         (None, 512)               0         
                                                                 
 dense_12 (Dense)            (None, 32)                16416     
                                                                 
 dense_13 (Dense)            (None, 10)                330       
                                                                 
Total params: 14,731,434
Trainable params: 12,995,946
Non-trainable params: 1,735,488
_________________________________________________________________


In [14]:
def run_model(model, batch_size=64, epochs=10):
    # Compile the model
    # adam optimizer: 
    # - adaptive learning rate: adjusts the step size for each parameter based on that parameter's performance
    # - momentum optimization: helps avoid local minima, achieve faster convergence
    # categorical crossentropy: 
    # - takes the predicted probabilities and the true class labels and calculates a single number that 
    #   represents the dissimilarity between them
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    # batch_size:
    # - batches refer to subsets of the training data that are processed together during the training of a neural network. 
    #   Instead of feeding all the training examples at once, the data is divided into smaller groups or batches, and the 
    #   model is updated based on the average gradients calculated from each batch.
    # - Standard practice is to choose a power-of-2 batch size
    # epoch:
    # - a complete iteration over the entire training dataset
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
    return model

# Final Model Comparison

In [22]:
vgg16_model_trained = run_model(vgg16_model)
base_model_trained = run_model(base_model)

# Evaluate base model
base_model_loss, base_model_accuracy = base_model_trained.evaluate(X_test, y_test)
print("Base Model:")
print(f"Loss: {base_model_loss:.4f}")
print(f"Accuracy: {base_model_accuracy:.4f}")

# Evaluate VGG16 model
vgg16_model_loss, vgg16_model_accuracy = vgg16_model_trained.evaluate(X_test, y_test)
print("VGG16 Model:")
print(f"Loss: {vgg16_model_loss:.4f}")
print(f"Accuracy: {vgg16_model_accuracy:.4f}")

# Compare model performances
print("\nModel Performance Comparison:")
if base_model_accuracy > vgg16_model_accuracy:
    print("Base model performs better.")
elif base_model_accuracy < vgg16_model_accuracy:
    print("VGG16 model performs better.")
else:
    print("Both models have the same accuracy.")

Epoch 1/10
782/782 [==============================] - 57s 30ms/step - loss: 1.5324 - accuracy: 0.4579 - val_loss: 1.0134 - val_accuracy: 0.6654
Epoch 2/10
782/782 [==============================] - 22s 28ms/step - loss: 0.9003 - accuracy: 0.7035 - val_loss: 0.8526 - val_accuracy: 0.7160
Epoch 3/10
782/782 [==============================] - 23s 29ms/step - loss: 0.7532 - accuracy: 0.7512 - val_loss: 0.7683 - val_accuracy: 0.7534
Epoch 4/10
782/782 [==============================] - 23s 29ms/step - loss: 0.6704 - accuracy: 0.7785 - val_loss: 0.7763 - val_accuracy: 0.7496
Epoch 5/10
782/782 [==============================] - 23s 29ms/step - loss: 0.6019 - accuracy: 0.8000 - val_loss: 0.7405 - val_accuracy: 0.7585
Epoch 6/10
782/782 [==============================] - 22s 28ms/step - loss: 0.5449 - accuracy: 0.8173 - val_loss: 0.7428 - val_accuracy: 0.7637
Epoch 7/10
782/782 [==============================] - 23s 29ms/step - loss: 0.4831 - accuracy: 0.8390 - val_loss: 0.7332 - val_accuracy:

# Model Experimentation

## This model seems to be severely overfitting
- next steps are to try fewer neurons in dense layers, and use dropout layers

3 layers unfrozen, 512, 256

In [ ]:
vgg16_model_trained = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 22s 24ms/step - loss: 1.1360 - accuracy: 0.6027 - val_loss: 0.9753 - val_accuracy: 0.6688
Epoch 2/10
782/782 [==============================] - 17s 21ms/step - loss: 0.8236 - accuracy: 0.7207 - val_loss: 0.8540 - val_accuracy: 0.7122
Epoch 3/10
782/782 [==============================] - 17s 21ms/step - loss: 0.7117 - accuracy: 0.7561 - val_loss: 0.8064 - val_accuracy: 0.7252
Epoch 4/10
782/782 [==============================] - 16s 21ms/step - loss: 0.6290 - accuracy: 0.7843 - val_loss: 0.8220 - val_accuracy: 0.7271
Epoch 5/10
782/782 [==============================] - 17s 21ms/step - loss: 0.5653 - accuracy: 0.8049 - val_loss: 0.7619 - val_accuracy: 0.7492
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.5037 - accuracy: 0.8243 - val_loss: 0.7962 - val_accuracy: 0.7374
Epoch 7/10
782/782 [==============================] - 17s 22ms/step - loss: 0.4425 - accuracy: 0.8463 - val_loss: 0.8379 - val_accuracy:

3 layers unfrozen, 64, 32

In [ ]:
vgg16_3_64_32 = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 31s 23ms/step - loss: 1.1202 - accuracy: 0.6105 - val_loss: 0.9468 - val_accuracy: 0.6763
Epoch 2/10
782/782 [==============================] - 18s 23ms/step - loss: 0.8183 - accuracy: 0.7198 - val_loss: 0.8065 - val_accuracy: 0.7262
Epoch 3/10
782/782 [==============================] - 16s 21ms/step - loss: 0.7081 - accuracy: 0.7573 - val_loss: 0.8719 - val_accuracy: 0.7081
Epoch 4/10
782/782 [==============================] - 16s 21ms/step - loss: 0.6334 - accuracy: 0.7817 - val_loss: 0.8323 - val_accuracy: 0.7268
Epoch 5/10
782/782 [==============================] - 17s 22ms/step - loss: 0.5638 - accuracy: 0.8056 - val_loss: 0.8801 - val_accuracy: 0.7180
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4958 - accuracy: 0.8284 - val_loss: 0.8697 - val_accuracy: 0.7290
Epoch 7/10
782/782 [==============================] - 17s 22ms/step - loss: 0.4473 - accuracy: 0.8437 - val_loss: 0.8312 - val_accuracy:

3 layers unfrozen, 64 - just one dense layer performs better

In [ ]:
vgg16_3_64 = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 20s 23ms/step - loss: 1.1004 - accuracy: 0.6163 - val_loss: 0.9438 - val_accuracy: 0.6795
Epoch 2/10
782/782 [==============================] - 17s 21ms/step - loss: 0.8043 - accuracy: 0.7252 - val_loss: 0.8067 - val_accuracy: 0.7280
Epoch 3/10
782/782 [==============================] - 17s 22ms/step - loss: 0.6959 - accuracy: 0.7605 - val_loss: 0.8134 - val_accuracy: 0.7223
Epoch 4/10
782/782 [==============================] - 17s 21ms/step - loss: 0.6159 - accuracy: 0.7856 - val_loss: 0.8064 - val_accuracy: 0.7259
Epoch 5/10
782/782 [==============================] - 16s 21ms/step - loss: 0.5403 - accuracy: 0.8104 - val_loss: 0.7863 - val_accuracy: 0.7400
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4734 - accuracy: 0.8326 - val_loss: 0.8132 - val_accuracy: 0.7385
Epoch 7/10
782/782 [==============================] - 18s 23ms/step - loss: 0.4173 - accuracy: 0.8528 - val_loss: 0.8924 - val_accuracy:

3 layers unfrozen, 64, dropout

In [ ]:
vgg16_3_64_dropout = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 19s 21ms/step - loss: 1.2535 - accuracy: 0.5713 - val_loss: 0.9121 - val_accuracy: 0.6933
Epoch 2/10
782/782 [==============================] - 17s 22ms/step - loss: 0.8714 - accuracy: 0.7076 - val_loss: 0.8619 - val_accuracy: 0.7049
Epoch 3/10
782/782 [==============================] - 17s 22ms/step - loss: 0.7623 - accuracy: 0.7458 - val_loss: 0.8236 - val_accuracy: 0.7302
Epoch 4/10
782/782 [==============================] - 17s 22ms/step - loss: 0.6929 - accuracy: 0.7686 - val_loss: 0.7989 - val_accuracy: 0.7358
Epoch 5/10
782/782 [==============================] - 18s 23ms/step - loss: 0.6168 - accuracy: 0.7919 - val_loss: 0.7957 - val_accuracy: 0.7426
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.5609 - accuracy: 0.8079 - val_loss: 0.8270 - val_accuracy: 0.7420
Epoch 7/10
782/782 [==============================] - 16s 21ms/step - loss: 0.5084 - accuracy: 0.8269 - val_loss: 0.8266 - val_accuracy:

3 layers unfrozen, 256, 64,  dropout



In [ ]:
vgg16_3_256_64_dropout = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 20s 22ms/step - loss: 1.2489 - accuracy: 0.5750 - val_loss: 0.9132 - val_accuracy: 0.6929
Epoch 2/10
782/782 [==============================] - 18s 22ms/step - loss: 0.9038 - accuracy: 0.7038 - val_loss: 0.8600 - val_accuracy: 0.7129
Epoch 3/10
782/782 [==============================] - 16s 21ms/step - loss: 0.7912 - accuracy: 0.7395 - val_loss: 0.8978 - val_accuracy: 0.7112
Epoch 4/10
782/782 [==============================] - 17s 22ms/step - loss: 0.7167 - accuracy: 0.7645 - val_loss: 0.8235 - val_accuracy: 0.7276
Epoch 5/10
782/782 [==============================] - 17s 22ms/step - loss: 0.6472 - accuracy: 0.7869 - val_loss: 0.8364 - val_accuracy: 0.7430
Epoch 6/10
782/782 [==============================] - 17s 21ms/step - loss: 0.5853 - accuracy: 0.8060 - val_loss: 0.8863 - val_accuracy: 0.7241
Epoch 7/10
782/782 [==============================] - 18s 23ms/step - loss: 0.5382 - accuracy: 0.8202 - val_loss: 0.8720 - val_accuracy:

## Trying to unfreeze a few more layers
- Unfreezing layers helps adjust the model to the task at hand

5 layers unfrozen 64 dropout .5

In [7]:
vgg16_5_64_dropout_50 = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 31s 23ms/step - loss: 1.3347 - accuracy: 0.5485 - val_loss: 0.9936 - val_accuracy: 0.6702
Epoch 2/10
782/782 [==============================] - 17s 22ms/step - loss: 0.9673 - accuracy: 0.6905 - val_loss: 0.8804 - val_accuracy: 0.7067
Epoch 3/10
782/782 [==============================] - 18s 22ms/step - loss: 0.8459 - accuracy: 0.7253 - val_loss: 0.8645 - val_accuracy: 0.7180
Epoch 4/10
782/782 [==============================] - 17s 21ms/step - loss: 0.7720 - accuracy: 0.7501 - val_loss: 0.8413 - val_accuracy: 0.7284
Epoch 5/10
782/782 [==============================] - 17s 21ms/step - loss: 0.7107 - accuracy: 0.7677 - val_loss: 0.8736 - val_accuracy: 0.7269
Epoch 6/10
782/782 [==============================] - 17s 21ms/step - loss: 0.6523 - accuracy: 0.7858 - val_loss: 0.8567 - val_accuracy: 0.7369
Epoch 7/10
782/782 [==============================] - 17s 22ms/step - loss: 0.6101 - accuracy: 0.8007 - val_loss: 0.8614 - val_accuracy:

8 layers unfrozen 64 dropout .25

In [ ]:
vgg16_8_64_dropout = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 26s 29ms/step - loss: 1.5949 - accuracy: 0.4002 - val_loss: 1.1644 - val_accuracy: 0.5783
Epoch 2/10
782/782 [==============================] - 23s 30ms/step - loss: 0.9839 - accuracy: 0.6726 - val_loss: 0.9032 - val_accuracy: 0.7074
Epoch 3/10
782/782 [==============================] - 23s 30ms/step - loss: 0.8327 - accuracy: 0.7295 - val_loss: 0.8631 - val_accuracy: 0.7257
Epoch 4/10
782/782 [==============================] - 22s 28ms/step - loss: 0.7468 - accuracy: 0.7545 - val_loss: 0.7888 - val_accuracy: 0.7477
Epoch 5/10
782/782 [==============================] - 23s 30ms/step - loss: 0.6850 - accuracy: 0.7762 - val_loss: 0.8613 - val_accuracy: 0.7384
Epoch 6/10
782/782 [==============================] - 22s 29ms/step - loss: 0.6432 - accuracy: 0.7902 - val_loss: 0.8220 - val_accuracy: 0.7474
Epoch 7/10
782/782 [==============================] - 23s 30ms/step - loss: 0.5948 - accuracy: 0.8039 - val_loss: 0.7624 - val_accuracy:

8 layers unfrozen 256 dropout .5

In [8]:
vgg16_8_256_dropout_25 = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 21s 23ms/step - loss: 0.4729 - accuracy: 0.8462 - val_loss: 1.0550 - val_accuracy: 0.7310
Epoch 2/10
782/782 [==============================] - 17s 21ms/step - loss: 0.4241 - accuracy: 0.8607 - val_loss: 1.1161 - val_accuracy: 0.7430
Epoch 3/10
782/782 [==============================] - 17s 21ms/step - loss: 0.3952 - accuracy: 0.8673 - val_loss: 1.1673 - val_accuracy: 0.7216
Epoch 4/10
782/782 [==============================] - 17s 21ms/step - loss: 0.3685 - accuracy: 0.8779 - val_loss: 1.1250 - val_accuracy: 0.7360
Epoch 5/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3373 - accuracy: 0.8885 - val_loss: 1.1784 - val_accuracy: 0.7282
Epoch 6/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3193 - accuracy: 0.8951 - val_loss: 1.2014 - val_accuracy: 0.7364
Epoch 7/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3009 - accuracy: 0.9013 - val_loss: 1.3999 - val_accuracy:

8 layers unfrozen 64 32 dropout .5

In [61]:
vgg16_8_64_32_dropout = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 27s 30ms/step - loss: 1.8858 - accuracy: 0.2723 - val_loss: 1.3586 - val_accuracy: 0.4862
Epoch 2/10
782/782 [==============================] - 23s 30ms/step - loss: 1.3733 - accuracy: 0.4837 - val_loss: 1.1805 - val_accuracy: 0.5773
Epoch 3/10
782/782 [==============================] - 23s 30ms/step - loss: 1.1978 - accuracy: 0.5820 - val_loss: 1.0251 - val_accuracy: 0.6558
Epoch 4/10
782/782 [==============================] - 23s 30ms/step - loss: 1.0415 - accuracy: 0.6583 - val_loss: 0.8715 - val_accuracy: 0.7145
Epoch 5/10
782/782 [==============================] - 23s 30ms/step - loss: 0.9330 - accuracy: 0.7009 - val_loss: 0.8824 - val_accuracy: 0.7173
Epoch 6/10
782/782 [==============================] - 23s 30ms/step - loss: 0.8762 - accuracy: 0.7207 - val_loss: 0.8475 - val_accuracy: 0.7382
Epoch 7/10
782/782 [==============================] - 22s 29ms/step - loss: 0.8129 - accuracy: 0.7411 - val_loss: 0.8356 - val_accuracy:

8 layers unfrozen 32 dropout .25

In [10]:
vgg16_8_32_dropout25 = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 28s 32ms/step - loss: 1.6481 - accuracy: 0.3875 - val_loss: 1.1421 - val_accuracy: 0.5754
Epoch 2/10
782/782 [==============================] - 24s 31ms/step - loss: 1.1139 - accuracy: 0.6188 - val_loss: 1.0032 - val_accuracy: 0.6386
Epoch 3/10
782/782 [==============================] - 22s 29ms/step - loss: 0.9179 - accuracy: 0.6957 - val_loss: 0.9040 - val_accuracy: 0.7019
Epoch 4/10
782/782 [==============================] - 23s 30ms/step - loss: 0.8130 - accuracy: 0.7353 - val_loss: 0.8134 - val_accuracy: 0.7375
Epoch 5/10
782/782 [==============================] - 23s 30ms/step - loss: 0.7377 - accuracy: 0.7632 - val_loss: 0.7453 - val_accuracy: 0.7589
Epoch 6/10
782/782 [==============================] - 22s 29ms/step - loss: 0.6796 - accuracy: 0.7809 - val_loss: 0.8026 - val_accuracy: 0.7455
Epoch 7/10
782/782 [==============================] - 23s 29ms/step - loss: 0.6264 - accuracy: 0.7982 - val_loss: 0.7588 - val_accuracy:

8 layers unfrozen 32 - this simpler model with only one dense layer seems to have an ideal level of complexity
- ultimate solution will likely have one dense layer and 8 layers unfrozen

In [16]:
vgg16_8_32 = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 26s 29ms/step - loss: 1.3869 - accuracy: 0.4990 - val_loss: 0.9737 - val_accuracy: 0.6813
Epoch 2/10
782/782 [==============================] - 22s 29ms/step - loss: 0.8574 - accuracy: 0.7085 - val_loss: 0.8326 - val_accuracy: 0.7226
Epoch 3/10
782/782 [==============================] - 23s 30ms/step - loss: 0.7310 - accuracy: 0.7529 - val_loss: 0.7529 - val_accuracy: 0.7539
Epoch 4/10
782/782 [==============================] - 23s 29ms/step - loss: 0.6477 - accuracy: 0.7829 - val_loss: 0.7513 - val_accuracy: 0.7562
Epoch 5/10
782/782 [==============================] - 22s 28ms/step - loss: 0.5845 - accuracy: 0.8037 - val_loss: 0.7734 - val_accuracy: 0.7541
Epoch 6/10
782/782 [==============================] - 23s 30ms/step - loss: 0.5433 - accuracy: 0.8179 - val_loss: 0.7241 - val_accuracy: 0.7749
Epoch 7/10
782/782 [==============================] - 23s 30ms/step - loss: 0.4895 - accuracy: 0.8353 - val_loss: 0.7616 - val_accuracy:

## Trying low, medium, and high regularization

8 layers unfrozen 32 with regularizer .01 - Best results yet!

In [7]:
# regularizer .01
vgg_8_32_regularizer = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 40s 31ms/step - loss: 1.5313 - accuracy: 0.4639 - val_loss: 1.0590 - val_accuracy: 0.6590
Epoch 2/10
782/782 [==============================] - 22s 28ms/step - loss: 0.8818 - accuracy: 0.7094 - val_loss: 0.8078 - val_accuracy: 0.7387
Epoch 3/10
782/782 [==============================] - 23s 29ms/step - loss: 0.7325 - accuracy: 0.7592 - val_loss: 0.8004 - val_accuracy: 0.7353
Epoch 4/10
782/782 [==============================] - 23s 29ms/step - loss: 0.6510 - accuracy: 0.7869 - val_loss: 0.7327 - val_accuracy: 0.7554
Epoch 5/10
782/782 [==============================] - 22s 29ms/step - loss: 0.5819 - accuracy: 0.8081 - val_loss: 0.7768 - val_accuracy: 0.7590
Epoch 6/10
782/782 [==============================] - 22s 28ms/step - loss: 0.5198 - accuracy: 0.8292 - val_loss: 0.7270 - val_accuracy: 0.7699
Epoch 7/10
782/782 [==============================] - 23s 29ms/step - loss: 0.4739 - accuracy: 0.8436 - val_loss: 0.7452 - val_accuracy:

8 layers unfrozen 32 with regularizer .01, dropout .5

In [20]:
# regularizer .01, dropout .5
vgg_8_32_regularizer01_dropout = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 27s 29ms/step - loss: 1.9589 - accuracy: 0.2479 - val_loss: 1.4546 - val_accuracy: 0.4691
Epoch 2/10
782/782 [==============================] - 23s 29ms/step - loss: 1.4613 - accuracy: 0.4488 - val_loss: 1.1527 - val_accuracy: 0.5877
Epoch 3/10
782/782 [==============================] - 23s 30ms/step - loss: 1.2323 - accuracy: 0.5641 - val_loss: 1.0694 - val_accuracy: 0.6407
Epoch 4/10
782/782 [==============================] - 23s 30ms/step - loss: 1.0985 - accuracy: 0.6272 - val_loss: 0.9693 - val_accuracy: 0.6856
Epoch 5/10
782/782 [==============================] - 22s 29ms/step - loss: 1.0125 - accuracy: 0.6608 - val_loss: 0.9514 - val_accuracy: 0.7062
Epoch 6/10
782/782 [==============================] - 23s 30ms/step - loss: 0.9225 - accuracy: 0.6974 - val_loss: 0.9769 - val_accuracy: 0.7176
Epoch 7/10
782/782 [==============================] - 22s 28ms/step - loss: 0.8850 - accuracy: 0.7066 - val_loss: 0.9640 - val_accuracy:

8 layers unfrozen 32 with regularizer .001

In [8]:
# regularizer .001
vgg_8_32_regularizer_001 = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 37s 30ms/step - loss: 1.4152 - accuracy: 0.4812 - val_loss: 1.1503 - val_accuracy: 0.6297
Epoch 2/10
782/782 [==============================] - 23s 29ms/step - loss: 0.8792 - accuracy: 0.7008 - val_loss: 0.8197 - val_accuracy: 0.7258
Epoch 3/10
782/782 [==============================] - 24s 30ms/step - loss: 0.7429 - accuracy: 0.7533 - val_loss: 0.8445 - val_accuracy: 0.7237
Epoch 4/10
782/782 [==============================] - 23s 29ms/step - loss: 0.6507 - accuracy: 0.7842 - val_loss: 0.7892 - val_accuracy: 0.7495
Epoch 5/10
782/782 [==============================] - 23s 29ms/step - loss: 0.5962 - accuracy: 0.8015 - val_loss: 0.7697 - val_accuracy: 0.7496
Epoch 6/10
782/782 [==============================] - 23s 29ms/step - loss: 0.5383 - accuracy: 0.8215 - val_loss: 0.6978 - val_accuracy: 0.7795
Epoch 7/10
782/782 [==============================] - 24s 31ms/step - loss: 0.4807 - accuracy: 0.8399 - val_loss: 0.7323 - val_accuracy:

8 layers unfrozen 32 with regularizer .1

In [10]:
# regularizer .1
vgg_8_32_regularizer_1 = run_model(vgg16_model)

Epoch 1/10
782/782 [==============================] - 27s 30ms/step - loss: 1.6962 - accuracy: 0.4754 - val_loss: 1.0526 - val_accuracy: 0.6563
Epoch 2/10
782/782 [==============================] - 23s 30ms/step - loss: 0.9035 - accuracy: 0.6997 - val_loss: 0.9595 - val_accuracy: 0.6935
Epoch 3/10
782/782 [==============================] - 24s 30ms/step - loss: 0.7519 - accuracy: 0.7536 - val_loss: 0.8143 - val_accuracy: 0.7306
Epoch 4/10
782/782 [==============================] - 23s 29ms/step - loss: 0.6682 - accuracy: 0.7808 - val_loss: 0.7586 - val_accuracy: 0.7556
Epoch 5/10
782/782 [==============================] - 24s 30ms/step - loss: 0.5913 - accuracy: 0.8065 - val_loss: 0.7430 - val_accuracy: 0.7654
Epoch 6/10
782/782 [==============================] - 24s 30ms/step - loss: 0.5306 - accuracy: 0.8265 - val_loss: 0.7526 - val_accuracy: 0.7710
Epoch 7/10
782/782 [==============================] - 24s 30ms/step - loss: 0.4639 - accuracy: 0.8480 - val_loss: 0.7401 - val_accuracy: